In [57]:
# DES Tables
IP = [58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6, 64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1, 59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7]

FP = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15, 55, 23, 63, 31,
      38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45, 13, 53, 21, 61, 29,
      36, 4, 44, 12, 52, 20, 60, 28, 35, 3, 43, 11, 51, 19, 59, 27,
      34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25]

E = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21, 20, 21, 22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]

P = [16, 7, 20, 21, 29, 12, 28, 17, 1, 15, 23, 26, 5, 18, 31, 10,
     2, 8, 24, 14, 32, 27, 3, 9, 19, 13, 30, 6, 22, 11, 4, 25]

S_BOXES = [
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]

# 16 round keys (48 bits each)
round_keys = ["010101010101010101010101010101010101010101010101" for _ in range(16)]

In [58]:
def des_round(left, right, round_key):
    # Expansion
    expanded = [right[E[i] - 1] for i in range(48)]
    print(f"Expanded: {''.join(map(str, expanded))}")

    # Key Mixing
    mixed = [int(expanded[i]) ^ int(round_key[i]) for i in range(48)]
    print(f"Key Mixed: {''.join(map(str, mixed))}")

    # Substitution
    substituted = []
    for i in range(8):
        block = mixed[i*6:(i+1)*6]
        row = int(f"{block[0]}{block[5]}", 2)
        col = int(''.join(map(str, block[1:5])), 2)
        val = S_BOXES[i][row][col]
        substituted.extend([int(bit) for bit in f"{val:04b}"])
    print(f"Substituted: {''.join(map(str, substituted))}")

    # Permutation
    permuted = [substituted[P[i] - 1] for i in range(32)]
    print(f"Permuted: {''.join(map(str, permuted))}")

    # XOR with left half
    new_right = [left[i] ^ permuted[i] for i in range(32)]
    print(f"New Right: {''.join(map(str, new_right))}")

    return right, new_right

In [59]:
def des_encrypt(block, round_keys):
    # Initial Permutation
    permuted = [block[IP[i] - 1] for i in range(64)]
    print(f"Initial Permutation: {''.join(map(str, permuted))}")

    left = permuted[:32]
    right = permuted[32:]

    for i in range(16):
        print(f"\nRound {i+1}:")
        left, right = des_round(left, right, round_keys[i])
        print(f"Left: {''.join(map(str, left))}")
        print(f"Right: {''.join(map(str, right))}")

    # Swap and Final Permutation
    combined = right + left
    ciphertext = [combined[FP[i] - 1] for i in range(64)]
    print(f"\nFinal Permutation: {''.join(map(str, ciphertext))}")

    return ciphertext

In [60]:
def cbc_encrypt(plaintext, iv, round_keys):
    # Split plaintext into 64-bit blocks
    blocks = [plaintext[i:i+64] for i in range(0, len(plaintext), 64)]

    # Pad the last block if necessary
    if len(blocks[-1]) < 64:
        blocks[-1] = blocks[-1].ljust(64, '0')

    ciphertext_blocks = []
    previous_block = iv

    for block in blocks:
        # XOR with previous ciphertext block (or IV for the first block)
        xor_block = [int(block[i]) ^ int(previous_block[i]) for i in range(64)]
        print(f"XOR Block: {''.join(map(str, xor_block))}")

        # DES encrypt
        encrypted_block = des_encrypt(xor_block, round_keys)
        ciphertext_blocks.append(encrypted_block)
        previous_block = encrypted_block

    return ciphertext_blocks

In [70]:
# Convert plaintext to binary representation (8-bit encoding per character)
def string_to_binary(input_string):
    """Convert a string into its binary representation (8 bits per character)."""
    return ''.join(format(ord(char), '08b') for char in input_string)


# Test case: Encrypt the phrase "Hello DES in CBC Mode"
plaintext = "DOBBY IS A FREE ELF"
binary_plaintext = string_to_binary(plaintext)  # Convert plaintext to binary
iv = '0' * 64  # Initialization Vector (IV) - 64-bit binary string
round_keys = ["010101010101010101010101010101010101010101010101" for _ in range(16)]  # Example round keys

# Encrypting the binary plaintext using CBC mode
ciphertext = cbc_encrypt(binary_plaintext, iv, round_keys)

# Printing the final ciphertext
print(f"\nFinal Ciphertext: {''.join(map(str, ciphertext))}")


XOR Block: 0100010001001111010000100100001001011001001000000100100101010011
Initial Permutation: 1101111110010000000000111101001000000000001000000101001010001110

Round 1:
Expanded: 000000000000000100000000001010100101010001011100
Key Mixed: 010101010101010001010101011111110000000100001001
Substituted: 11000001001000100110011100101010
Permuted: 00001000111001101110010010001100
New Right: 11010111011101101110011101011110
Left: 00000000001000000101001010001110
Right: 11010111011101101110011101011110

Round 2:
Expanded: 011010101110101110101101011100001110101011111101
Key Mixed: 001111111011111011111000001001011011111110101000
Substituted: 00010101010101010100101100101001
Permuted: 10011100001001010111110000010010
New Right: 10011100000001010010111010011100
Left: 11010111011101101110011101011110
Right: 10011100000001010010111010011100

Round 3:
Expanded: 010011111000000000001010100101011101010011111001
Key Mixed: 000110101101010101011111110000001000000110101100
Substituted: 00010100010110

In [82]:
def binary_to_text(binary_data):
    """
    Converts binary data into a readable English text string.
    Args:
        binary_data: A list of binary digits (0s and 1s).
    Returns:
        A decoded string of plaintext.
    """
    # Group binary data into 8-bit chunks and convert to ASCII characters
    chars = [binary_data[i:i + 8] for i in range(0, len(binary_data), 8)]
    text = ''.join(chr(int(''.join(map(str, char)), 2)) for char in chars)
    return text


def xor_blocks(block1, block2):
    """
    XOR operation between two binary blocks.
    Args:
        block1: First binary block (list of 0s and 1s).
        block2: Second binary block (same length as block1).
    Returns:
        XOR-ed binary block.
    """
    return [b1 ^ b2 for b1, b2 in zip(block1, block2)]


def des_decrypt_single_block(block, round_keys):
    """
    DES decryption for a single 64-bit block.
    Args:
        block: The 64-bit binary ciphertext block.
        round_keys: List of 16 DES round keys (in reverse order for decryption).
    Returns:
        A 64-bit binary plaintext block.
    """
    # Reverse the round keys for decryption
    reversed_round_keys = round_keys[::-1]

    # Initial Permutation (IP)
    permuted = [block[IP[i] - 1] for i in range(64)]
    left, right = permuted[:32], permuted[32:]

    # Apply 16 rounds
    for round_key in reversed_round_keys:
        left, right = des_round(left, right, round_key)

    # Post-processing: Swap halves and apply Final Permutation (FP)
    preoutput = right + left
    plaintext_block = [preoutput[FP[i] - 1] for i in range(64)]

    return plaintext_block


def des_cbc_decrypt(ciphertext, round_keys, iv):
    """
    Decrypts ciphertext using DES in CBC mode.
    Args:
        ciphertext: Full binary ciphertext (multiple 64-bit blocks).
        round_keys: List of 16 DES round keys.
        iv: Initialization vector (64-bit binary block).
    Returns:
        Decrypted plaintext as an English-readable string.
    """
    block_size = 64  # DES block size
    plaintext_binary = []
    previous_block = iv

    # Process each block
    for block_start in range(0, len(ciphertext), block_size):
        # Get the current ciphertext block
        current_block = ciphertext[block_start:block_start + block_size]

        # Decrypt the current block using DES
        decrypted_block = des_decrypt_single_block(current_block, round_keys)

        # XOR decrypted block with the previous ciphertext block (or IV for the first block)
        plaintext_block = xor_blocks(decrypted_block, previous_block)

        # Append to the final plaintext
        plaintext_binary.extend(plaintext_block)

        # Update the previous block for the next iteration
        previous_block = current_block

    # Convert binary plaintext to ASCII text
    plaintext = binary_to_text(plaintext_binary)
    return plaintext


# Example usage
if __name__ == "__main__":
    # Ciphertext (192 bits — 3 blocks of 64 bits)
    ciphertext = [
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,  # Block 1
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,  # Block 2
        0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,  # Block 3
        1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1
    ]

    # Example round keys (mock data — usually generated using a key schedule)
    round_keys = ["010101010101010101010101010101010101010101010101" for _ in range(16)]

    # Initialization vector (IV) for CBC mode (64-bit block)
    iv = [
        1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0
    ]

    # Decrypt the ciphertext
    plaintext = des_cbc_decrypt(ciphertext, round_keys, iv)
    print(f"\nDecrypted Plaintext: {plaintext}")


TypeError: des_cbc_decrypt() missing 1 required positional argument: 'iv'